In [16]:
from ollama import chat, ChatResponse
import json
import re
import time

# Axiomatic symbol set (expanded to include all referenced characters)
axiomatic_symbols = {
    "CHARACTERS": ["A", "B", "C", "D", "E", "F"],
    "ACTIONS": ["LOVE", "KILL", "WANT", "OWN", "SAVE", "LEAVE", "DIE", "FIGHT", "RULE", "FLEE", "PROTECT", "FOLLOW", "REBEL", "TRAIN"],
    "OBJECTS": ["THRONE", "LIFE", "KINGDOM", "SPICE", "DESTINY", "FAMILY", "PLANET", "POWER"],
    "EMOTIONS": ["GRIEF", "FEAR", "HOPE", "JOY", "SADNESS", "ANGER", "GUILT", "HATE", "AMBITION"],
    "RELATIONS": ["FATHER", "MOTHER", "SON", "FRIEND", "ENEMY", "LOVER", "MENTOR"],
    "MODIFIERS": ["NOT", "BEFORE", "AFTER", "CAUSES", "→", "=", "IF", "WHEN"]
}

# Tracks all derived symbols
derived_symbols = {}

# Store final compression for each fragment
final_compressions = []

# A sequence of Dune synopsis fragments
dune_fragments = [
    "A is the son of B. B is the ruler of planet Arrakis. B is betrayed and killed by C.",
    "A and D flee into the desert. D is A's mother. They survive with help from E.",
    "A learns about his destiny to lead. He begins to have visions of the future.",
    "A trains with E's people and earns their respect.",
    "C continues to rule with cruelty. A plans rebellion.",
    "A fights and defeats C. A becomes ruler of Arrakis.",
    "A marries F, a princess. But he still loves E.",
    "A struggles with guilt and visions of future destruction.",
    "A accepts his role but knows it comes with great sacrifice.",
]

def build_prompt(axioms, derived, story):
    lines = [
        "You are a symbolic compression engine.",
        "Each iteration, you must represent the given synopsis using only the axiomatic and previously defined derived symbols.",
        "You may only define a new derived symbol if it is made purely from already available symbols.",
        "",
        "AXIOMATIC + DERIVED SYMBOLS:"
    ]
    
    for category, symbols in axioms.items():
        lines.append(f"# {category}")
        for s in symbols:
            lines.append(f"- {s}")
    lines.append("")
    
    lines.append("# DERIVED SYMBOLS")
    if not derived:
        lines.append("- None")
    else:
        for k, v in derived.items():
            lines.append(f"- {k} := {v}")
    lines.append("")
    
    lines.append("TASK:")
    lines.append(f"Compress the following scene:")
    lines.append(f"\"{story}\"\n")
    
    lines.append("Output FORMAT:")
    lines.append("- COMPRESSED: [symbolic form]")
    lines.append("- NEW SYMBOLS: [only list newly created symbols]")
    lines.append("  - name := definition")
    
    return "\n".join(lines)

def send_to_ollama(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            response: ChatResponse = chat(
                model='llama3.1',  # Change to appropriate model if needed
                messages=[
                    {
                        'role': 'user',
                        'content': prompt,
                    }
                ]
            )
            return response.message.content
        except Exception as e:
            print(f"API Error (attempt {attempt+1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait before retrying
            else:
                raise

def parse_response(text):
    # Extract the compressed representation
    compressed_match = re.search(r"COMPRESSED:\s*(.+?)(?:\n|$)", text, re.DOTALL)
    compressed = compressed_match.group(1).strip() if compressed_match else None
    
    # Extract new symbol definitions
    new_defs = {}
    new_symbols_section = re.search(r"NEW SYMBOLS:(.+?)(?:\n\n|$)", text, re.DOTALL)
    
    if new_symbols_section:
        symbol_text = new_symbols_section.group(1)
        for line in symbol_text.strip().splitlines():
            if ":=" in line:
                parts = line.strip().split(":=", 1)
                if len(parts) == 2:
                    key = parts[0].strip("- ").strip()
                    val = parts[1].strip()
                    new_defs[key] = val
    
    return compressed, new_defs

# === MAIN COMPRESSION LOOP ===
print("🚀 Starting symbolic compression of Dune\n")
all_outputs = []
iteration = 1

for fragment_index, story in enumerate(dune_fragments):
    print(f"\n====== PROCESSING FRAGMENT {fragment_index + 1} ======")
    print(f"Text: \"{story}\"")
    
    last_compressed = None
    unchanged_count = 0
    
    for i in range(10):  # Max iterations per fragment (reduced from 50)
        print(f"\n--- ITERATION {iteration} ---")
        prompt = build_prompt(axiomatic_symbols, derived_symbols, story)
        
        try:
            response = send_to_ollama(prompt)
            print(response)
            
            # Save the response for later review
            all_outputs.append(f"ITERATION {iteration}:\n{response}\n")
            
            # Parse the response
            compressed, new_defs = parse_response(response)
            
            if compressed == last_compressed:
                unchanged_count += 1
            else:
                unchanged_count = 0
                last_compressed = compressed
            
            # If we've reached convergence (no new symbols or unchanged for 2 iterations)
            if not new_defs or unchanged_count >= 2:
                print(f"✅ Final compression form reached: {compressed}")
                final_compressions.append(compressed)
                break
                
            # Add new derived symbols
            derived_symbols.update(new_defs)
            
        except Exception as e:
            print(f"Error in iteration {iteration}: {e}")
        
        iteration += 1

print("\n🧠 FINAL DERIVED SYMBOLS:\n")
for name, definition in derived_symbols.items():
    print(f"{name} := {definition}")

print("\n📝 FINAL COMPRESSED NARRATIVE:\n")
for i, compression in enumerate(final_compressions):
    print(f"Fragment {i+1}: {compression}")

# Attempt to create a unified compressed representation
if final_compressions:
    unified_story = " → ".join(final_compressions)
    print("\n🎬 UNIFIED STORY COMPRESSION:")
    print(unified_story)

🚀 Starting symbolic compression of Dune


====== PROCESSING FRAGMENT 1 ======
Text: "A is the son of B. B is the ruler of planet Arrakis. B is betrayed and killed by C."

--- ITERATION 1 ---
# COMPRESSED
COMPRESSED: A = SON(B), B = RULE(PLANET, PLANET::Arrakis) → KILL(C)

# NEW SYMBOLS:
- NAME := RULE(definition)
  - NAME := RULER OF (OBJECT::)
- OBJECT ::= PLANET 
  - (new axiom)

--- ITERATION 2 ---
**Iteration 1**

To compress the scene, I'll start by replacing the given text with its symbolic representation.

"A is the son of B." → SON = FRIEND (FATHER:: B)
"B is the ruler of planet Arrakis." → RULER = NAME (OBJECT :: PLANET:: Arrakis)
"B is betrayed and killed by C." → BETRAYED = → KILL (LOVER / ENEMY:: C)

**COMPRESSED:** [SON = FRIEND (FATHER:: B) AND RULER = NAME (OBJECT :: PLANET:: Arrakis) CAUSES BETRAYED = → KILL (LOVER / ENEMY:: C)]
**NEW SYMBOLS:**
- name := definition
  - RULER := NAME (OBJECT ::= PLANET)
  - SON := FRIEND (FATHER ::)

Note that I've only used the given a

In [17]:
derived_symbols

{'NAME': 'RULER OF (OBJECT::)',
 'OBJECT :': 'PLANET',
 'name': 'definition',
 'RULER': 'NAME (OBJECT ::= PLANET)',
 'SON': 'FRIEND (FATHER ::)',
 'MOTHER': 'RULER OF (FRIEND = RULER OF OBJECT ::= PLANET)',
 '* PEOPLE': 'OBJECT : = PLANET',
 '* TRAIN': 'ACTION (OBJECT ::= PLANET, FRIEND ::= SON)',
 '* RESPECT': 'EMOTION (FRIEND = RULER OF OBJECT ::= PLANET)',
 '* BUT': 'NOT AND',
 '* AND': 'IF (THEN (A, B), THEN (B, C))',
 '* FUTURE DESTRUCTION': 'VISION OF DESTINY',
 '* VISION': 'OBJECT (EMOTION = GUILT)',
 '* DESTINY': 'LIFE (DESTROYED BY ACTION = KILL OR ACTION = FIGHT)',
 'STRUGGLE': 'RULER RESIST (OBJECT ::= FUTURE DESTRUCTION)'}